# **Wageningen B-Series Propellers** 

#### **Parameters:**
- **Aᴇ/A₀**: Blade area ratio
- **C(Q) s,t,u,v**: Coefficient in the \(K_Q\) polynomial expression
- **C(T) s,t,u,v**: Coefficient in the \(K_T\) polynomial expression
- **D**: Propeller diameter
- **J**: Advance coefficient
- **K_Q**: Torque coefficient
- **K_T**: Thrust coefficient
- **n**: Propeller revolutions per second
- **P/D**: Pitch-diameter ratio
- **Q**: Propeller torque
- **Re**: Reynolds number
- **T**: Propeller thrust
- **t/c**: Thickness to cord ratio for propeller blades
- **V_A**: Speed of advance
- **Z**: Number of blades
- **η**: Open-water propeller efficiency
- **ρ**: Fluid density

#### **Parameters Required for KT KQ calculation:**
1. **J**, Advanced ratio (float)
  Formula: J = Va/nD
  VA Formula: VA = V⋅(1-wT)⋅0.5144

3. **P/D**, Blade area ratio (float)
  Formula: P/D


5. **Aᴇ/A₀**, Blade area ratio (float)
  Formula: - Aᴇ/A₀


7. **z**, Number of Blades (int)


**Calculate KT**: KT = ∑C s,t,u,v⋅(J^s)⋅((P/D)^t)⋅((Aᴇ/A₀)^u)⋅((z)^v)

**Calculate KQ**: KQ = ∑C s,t,u,v⋅(J^s)⋅((P/D)^t)⋅((Aᴇ/A₀)^u)⋅((z)^v)




#### **User inputs**:
1. V, speed in knots (int)
2. wT, wake fraction coefficient (float)
3. P, pitch in meters (float)
4. D, diameter in meters (float)
5. BAR or Aᴇ/A₀ , blade area ratio (float)
6. z, number of blades (int)


# **Libraries**

In [1]:
import matplotlib
matplotlib.use('qtagg')  # Or 'qt5agg' for Qt-based backends
%matplotlib qt 

# Import libraries
import tkinter as tk
from tkinter import simpledialog, messagebox
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import mplcursors
import matplotlib.cm as cm
import numpy as np
from math import pi, log10



# **Calculations**

In [2]:
class PropellerCalculator:
    def __init__(self):
        # Coefficients for K_T from the table
        self.KT_coefficients = [
            [0.00880496, 0, 0, 0, 0],
            [-0.204554, 1, 0, 0, 0],
            [0.166351, 0, 1, 0, 0],
            [0.158114, 0, 2, 0, 0],
            [-0.147581, 2, 0, 1, 0],
            [-0.481497, 1, 1, 1, 0],
            [0.415437, 0, 2, 1, 0],
            [0.0144043, 0, 0, 0, 1],
            [-0.0530054, 2, 0, 0, 1],
            [0.0143481, 0, 1, 0, 1],
            [0.0606826, 1, 1, 0, 1],
            [-0.0125894, 0, 0, 1, 1],
            [0.0109689, 1, 0, 1, 1],
            [-0.133698, 0, 3, 0, 0],
            [0.00638407, 0, 6, 0, 0],
            [-0.00132718, 2, 6, 0, 0],
            [0.168496, 3, 0, 1, 0],
            [-0.0507214, 0, 0, 2, 0],
            [0.0854559, 2, 0, 2, 0],
            [-0.0504475, 3, 0, 2, 0],
            [0.010465, 1, 6, 2, 0],
            [-0.00648272, 2, 6, 2, 0],
            [-0.00841728, 0, 3, 0, 1],
            [0.0168424, 1, 3, 0, 1],
            [-0.00102296, 3, 3, 0, 1],
            [-0.0317791, 0, 3, 1, 1],
            [0.018604, 1, 0, 2, 1],
            [-0.00410798, 0, 2, 2, 1],
            [-0.000606848, 0, 0, 0, 2],
            [-0.0049819, 1, 0, 0, 2],
            [0.0025983, 2, 0, 0, 2],
            [-0.000560528, 3, 0, 0, 2],
            [-0.00163652, 1, 2, 0, 2],
            [-0.000328787, 1, 6, 0, 2],
            [0.000116502, 2, 6, 0, 2],
            [0.000690904, 0, 0, 1, 2],
            [0.00421749, 0, 3, 1, 2],
            [0.0000565229, 3, 6, 1, 2],
            [-0.00146564, 0, 3, 2, 2],
        ]
        self.KQ_coefficients = [
            [0.00379368, 0, 0, 0, 0],
            [0.00886523, 2, 0, 0, 0],
            [-0.032241, 1, 1, 0, 0],
            [0.00344778, 0, 2, 0, 0],
            [-0.0408811, 0, 1, 1, 0],
            [-0.108009, 1, 1, 1, 0],
            [-0.0885381, 2, 1, 1, 0],
            [0.188561, 0, 2, 1, 0],
            [-0.00370871, 1, 0, 0, 1],
            [0.00513696, 0, 1, 0, 1],
            [0.0209449, 1, 1, 0, 1],
            [0.00474319, 2, 1, 0, 1],
            [-0.00723408, 2, 0, 1, 1],
            [0.00438388, 1, 1, 1, 1],
            [-0.0269403, 0, 2, 1, 1],
            [0.0558082, 3, 0, 1, 0],
            [0.0161886, 0, 3, 1, 0],
            [0.00318086, 1, 3, 1, 0],
            [0.015896, 0, 0, 2, 0],
            [0.0471729, 1, 0, 2, 0],
            [0.0196283, 3, 0, 2, 0],
            [-0.0502782, 0, 1, 2, 0],
            [-0.030055, 3, 1, 2, 0],
            [0.0417122, 2, 2, 2, 0],
            [-0.0397722, 0, 3, 2, 0],
            [-0.00350024, 0, 6, 2, 0],
            [-0.0106854, 3, 0, 0, 1],
            [0.00110903, 3, 3, 0, 1],
            [-0.000313912, 0, 6, 0, 1],
            [0.0035985, 3, 0, 1, 1],
            [-0.00142121, 0, 6, 1, 1],
            [-0.00383637, 1, 0, 2, 1],
            [0.0126803, 0, 2, 2, 1],
            [-0.00318278, 2, 3, 2, 1],
            [0.00334268, 0, 6, 2, 1],
            [-0.00183491, 1, 1, 0, 2],
            [0.000112451, 3, 2, 0, 2],
            [-0.0000297228, 3, 6, 0, 2],
            [0.000269551, 1, 0, 1, 2],
            [0.00083265, 2, 0, 1, 2],
            [0.00155334, 0, 2, 1, 2],
            [0.000302683, 0, 6, 1, 2],
            [-0.0001843, 0, 0, 2, 2],
            [-0.000425399, 0, 3, 2, 2],
            [0.0000869243, 3, 3, 2, 2],
            [-0.0004659, 0, 6, 2, 2],
            [0.0000554194, 1, 6, 2, 2],
        ]
    def calculate_advance_ratio(self, V, wT, D, n):
        """
        Calculate advance ratio (J).
        J = Va / (n * D)
        """
        Va = V * (1 - wT) * 0.514444  # Convert knots to m/s
        '''
        if n == 0:
            return 0  # J = 0 for static condition
        '''
        return Va / (n * D)

    def calculate_KT(self, J, P_D, AE_AO, z):
        """
        Calculate thrust coefficient (K_T) based on the polynomial equation.
        """
        KT = 0
        for coeff, s, t, u, v in self.KT_coefficients:
            KT += coeff * (J ** s) * (P_D ** t) * (AE_AO ** u) * (z ** v)
        return KT

    def calculate_KQ(self, J, P_D, AE_AO, z):
        """
        Calculate torque coefficient (K_Q) based on the polynomial equation.
        """
        KQ = 0
        for coeff, s, t, u, v in self.KQ_coefficients:
            KQ += coeff * (J ** s) * (P_D ** t) * (AE_AO ** u) * (z ** v)
        return KQ

    def calculate(self, V, wT, P, D, AE_AO, z, n):
        """
        Main calculation function.
        """
        P_D = P / D
        J = self.calculate_advance_ratio(V, wT, D, n)
        KT = self.calculate_KT(J, P_D, AE_AO, z)
        KQ = self.calculate_KQ(J, P_D, AE_AO, z)
        return KT, KQ, J


# ***CODE***

In [3]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mplcursors


class InputGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Propeller Input Parameters")

        # Create labels and entry fields
        self.entries = {}
        self.labels = {
            "wT": "Wake fraction (wT):",
            "D": "Propeller diameter (D in meters):",
            "Ae_Ao": "Blade area ratio (Ae/Ao):",
            "Z": "Number of blades (Z):",
            "rho": "Fluid density (ρ in kg/m³):",
            "V": "Vessel speed (V in knots):",
            "PD_array": "P/D ratios (comma-separated):",
        }
        self.default_values = {
            "wT": "0",
            "D": "1.2",
            "Ae_Ao": "0.4",
            "Z": "4",
            "rho": "1023.6",
            "V": "9",
            "PD_array": "0.5, 0.6, 0.7, 0.8",
        }

        row = 0
        for key, label in self.labels.items():
            tk.Label(self.root, text=label, anchor="w").grid(row=row, column=0, padx=10, pady=5, sticky="w")
            self.entries[key] = tk.Entry(self.root, width=30)
            self.entries[key].insert(0, self.default_values[key])
            self.entries[key].grid(row=row, column=1, padx=10, pady=5)
            row += 1

        # Add Submit button
        self.submit_button = tk.Button(self.root, text="Submit", command=self.submit)
        self.submit_button.grid(row=row, column=0, columnspan=2, pady=10)

        self.root.mainloop()

    def submit(self):
        try:
            # Get and validate user inputs
            self.wT = float(self.entries["wT"].get())
            self.D = float(self.entries["D"].get())
            self.Ae_Ao = float(self.entries["Ae_Ao"].get())
            self.Z = int(self.entries["Z"].get())
            self.rho = float(self.entries["rho"].get())
            self.V = float(self.entries["V"].get())
            self.PD_array = [float(x.strip()) for x in self.entries["PD_array"].get().split(",")]

            # Close the GUI window
            self.root.destroy()
        except ValueError as e:
            messagebox.showerror("Input Error", f"Invalid input: {e}")


# Main function to execute after getting user input
def main():
    # Prompt user for input
    gui = InputGUI()

    # Extract the values
    wT = gui.wT
    D = gui.D
    Ae_Ao = gui.Ae_Ao
    Z = gui.Z
    rho = gui.rho
    V = gui.V
    PD_array = gui.PD_array

    # Initialize PropellerCalculator (Assume it's already defined)
    propeller_calculator = PropellerCalculator()

    # Rotational speed range in RPS
    n_values = np.linspace(1, 1000, 6000)

    # Plot setup
    fig, ax1 = plt.subplots(figsize=(20, 10))
    ax2 = ax1.twinx()  # Second y-axis for KQ
    ax3 = ax1.twinx()  # Third y-axis for Efficiency
    ax3.spines["right"].set_position(("outward", 60))

    colors = cm.viridis(np.linspace(0, 1, len(PD_array)))  # Colormap for consistent colors

    hover_data_ax1 = []  # To store data for KT (ax1)
    hover_data_ax2 = []  # To store data for KQ (ax2)
    hover_data_ax3 = []  # To store data for Efficiency (ax3)

    # Loop through P/D ratios
    for i, PD in enumerate(PD_array):
        P = D * PD  # Calculate pitch for the current P/D ratio
        KT_values, KQ_values, J_values, efficiency_values = [], [], [], []

        for n in n_values:
            KT, KQ, J = propeller_calculator.calculate(V, wT, P, D, Ae_Ao, Z, n)
            KT_values.append(KT)
            KQ_values.append(KQ)
            J_values.append(J)
            efficiency = (KT / (KQ * 10)) * (J / (2 * np.pi)) if KQ > 0 else 0
            efficiency_values.append(efficiency)

        color = colors[i]
        line_kt, = ax1.plot(J_values, KT_values, label=f"KT (P/D={PD:.1f})", linestyle=":", color=color, linewidth=1)
        line_kq, = ax2.plot(J_values, KQ_values, label=f"KQ (P/D={PD:.1f})", linestyle="--", color=color, linewidth=1)
        line_eff, = ax3.plot(J_values, efficiency_values, label=f"Efficiency (P/D={PD:.1f})", linestyle="-", color=color, linewidth=1)

        hover_data_ax1.append({
            "line": line_kt,
            "J_values": J_values,
            "KT_values": KT_values,
            "P/D": PD,
            "n_values": n_values
        })
        hover_data_ax2.append({
            "line": line_kq,
            "J_values": J_values,
            "KQ_values": KQ_values,
            "P/D": PD,
            "n_values": n_values
        })
        hover_data_ax3.append({
            "line": line_eff,
            "J_values": J_values,
            "Efficiency_values": efficiency_values,
            "P/D": PD,
            "n_values": n_values
        })

    # Configure Y-axis: KT
    ax1.set_ylabel("KT (Thrust Coefficient)", color="blue")
    ax1.tick_params(axis="y", labelcolor="blue")
    ax1.set_ylim(0, 1)
    ax1.set_yticks(np.arange(0, 1.2, 0.2))

    # Configure Y-axis: KQ
    ax2.set_ylabel("KQ (Torque Coefficient)", color="green")
    ax2.tick_params(axis="y", labelcolor="green")
    ax2.set_ylim(0, 0.2)
    ax2.set_yticks(np.arange(0, 0.24, 0.04))

    # Configure Y-axis: Efficiency
    ax3.set_ylabel("Efficiency", color="orange")
    ax3.tick_params(axis="y", labelcolor="orange")
    ax3.set_ylim(0, 0.2)
    ax3.set_yticks(np.arange(0, 0.24, 0.04))

    # Configure X-axis
    ax1.set_xlabel("J (Advance Coefficient)")
    ax1.set_xlim(0, 1.4)
    ax1.set_xticks(np.arange(0, 1.6, 0.2))

    # Title and Grid
    plt.title(f"Propeller Coefficients and Efficiency at {V:.1f} knots")
    ax1.grid()

    # Combined legend
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    lines3, labels3 = ax3.get_legend_handles_labels()
    ax1.legend(lines + lines2 + lines3, labels + labels2 + labels3, loc="upper left", bbox_to_anchor=(0.01, 0.98))

    # Configure mplcursors for ax1, ax2, and ax3
    cursor_ax1 = mplcursors.cursor([d["line"] for d in hover_data_ax1], hover=True)
    cursor_ax2 = mplcursors.cursor([d["line"] for d in hover_data_ax2], hover=True)
    cursor_ax3 = mplcursors.cursor([d["line"] for d in hover_data_ax3], hover=True)
    
    @cursor_ax1.connect("add")
    def on_add_ax1(sel):
        label = sel.artist.get_label()
        n_value = None
    
        # Loop through hover_data_ax1 to find the matching line
        for data in hover_data_ax1:
            if sel.artist == data["line"]:  # Match the hovered line
                # Extract the closest J value
                J_values = data["J_values"]
                KT_values = data["KT_values"]
                # Use tolerance to find the closest J and KT
                tolerance = 1e-2
                for i, J in enumerate(J_values):
                    if abs(J - sel.target[0]) < tolerance and abs(KT_values[i] - sel.target[1]) < tolerance:
                        n_value = n_values[i]  # Get the corresponding RPS (n) value
                        break
                break
        Thrust = (sel.target[1])*(rho*(n_value**2)*(D**4))
        RPM = n_value*60
        # Update the annotation
        sel.annotation.set_text(
            f"$\\mathbf{{{label}}}$\n\nKT: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: {RPM:.2f}\nThrust(N): {Thrust:.2f}" if n_value else
            f"$\\mathbf{{{label}}}$\n\nKT: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: NA\nThrust(N): NA"
        )
    
    @cursor_ax2.connect("add")
    def on_add_ax2(sel):
        label = sel.artist.get_label()
        n_value = None
    
        # Loop through hover_data_ax2 to find the matching line
        for data in hover_data_ax2:
            if sel.artist == data["line"]:  # Match the hovered line
                # Extract the closest J value
                J_values = data["J_values"]
                KQ_values = data["KQ_values"]
    
                # Use tolerance to find the closest J and KQ
                tolerance = 1e-2
                for i, J in enumerate(J_values):
                    if abs(J - sel.target[0]) < tolerance and abs(KQ_values[i] - sel.target[1]) < tolerance:
                        n_value = n_values[i]  # Get the corresponding RPS (n) value
                        break
                break
                
        Torque = (sel.target[1])*(rho*(n_value**2)*(D**5))
        RPM = n_value*60
        
        # Update the annotation
        sel.annotation.set_text(
            f"$\\mathbf{{{label}}}$\n\nKQ: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: {RPM:.2f}\nTorque(Nm): {Torque:.2f}" if n_value else
            f"$\\mathbf{{{label}}}$\n\nKQ: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: NA\nTorque(Nm): NA"
        )
    
    @cursor_ax3.connect("add")
    def on_add_ax3(sel):
        label = sel.artist.get_label()
        n_value = None
    
        # Loop through hover_data_ax3 to find the matching line
        for data in hover_data_ax3:
            if sel.artist == data["line"]:  # Match the hovered line
                # Extract the closest J value
                J_values = data["J_values"]
                Efficiency_values = data["Efficiency_values"]
    
                # Use tolerance to find the closest J and Efficiency
                tolerance = 1e-2
                for i, J in enumerate(J_values):
                    if abs(J - sel.target[0]) < tolerance and abs(Efficiency_values[i] - sel.target[1]) < tolerance:
                        n_value = n_values[i]  # Get the corresponding RPS (n) value
                        break
                break

        RPM = n_value*60
        
        # Update the annotation
        sel.annotation.set_text(
            f"$\\mathbf{{{label}}}$\n\nEfficiency: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: {RPM:.2f}" if n_value else
            f"$\\mathbf{{{label}}}$\n\nEfficiency: {sel.target[1]:.3f}\nJ: {sel.target[0]:.3f}\nRPM: NA"
        )

    
    plt.show()


# Run the application
main()


# **BACKUP**